In [6]:
import pandas as pd
import re

In [7]:
products = pd.read_csv('./Dataset/Amazon_Products.csv')
market_cap = pd.read_csv('./Dataset/Market_Cap.csv')
revenue = pd.read_csv('./Dataset/Revenue.csv')

del products['Unnamed: 0']

In [8]:
df = pd.merge(market_cap, revenue, on='Name', how='inner').drop(['Rank_x', 'Rank_y', 'Symbol_y', 'country_y', 'price (USD)_y'], axis=1)

In [9]:
df.columns = ['brand', 'Symbol', 'marketcap', 'price (USD)', 'country', 'revenue_ttm']

In [10]:
df['brand'] = df['brand'].str.extract('\((.*?)\)', expand=False).fillna(df['brand'])
df['brand'] = df['brand'].str.lower().str.capitalize()

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/var/folders/bt/smkkfpmx06961p03z9_f5gyr0000gn/T/ipykernel_3266/1172336472.py:1: SyntaxWarning: invalid escape sequence '\('
  df['brand'] = df['brand'].str.extract('\((.*?)\)', expand=False).fillna(df['brand'])


## Pre-processing

In [11]:
# Removing the ₹ sign
products["discount_price"] = products["discount_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)
products["actual_price"] = products["actual_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)

# Change commas to dots and change the type to float
products['discount_price'] = products["discount_price"].str.replace(',', '').astype(float)
products["actual_price"] = products["actual_price"].str.replace(',', '').astype(float)

In [12]:
products['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '4.9', '2.2', '1.6', '1.9', '2.0',
       '1.4', '2.1', 'FREE', '1.2', '1.3', '1.5', '₹68.99', '₹65', '1.1',
       '₹70', '₹100', '₹99', '₹2.99'], dtype=object)

In [13]:
# Replace certain values in the 'ratings' column of the 'products' dataframe with '0.0'
products['ratings'] = products['ratings'].replace(['Get','FREE','₹68.99', '₹65','₹70', '₹100', '₹99', '₹2.99'], '0.0')
products['ratings'] = products["ratings"].astype(float)
products['ratings'].unique()

array([4.2, 4. , 4.1, 4.3, 3.9, 3.8, 3.5, nan, 4.6, 3.3, 3.4, 3.7, 2.9,
       5. , 4.4, 3.6, 2.7, 4.5, 3. , 3.1, 3.2, 4.8, 4.7, 2.5, 1. , 2.6,
       2.8, 2.3, 1.7, 0. , 1.8, 2.4, 4.9, 2.2, 1.6, 1.9, 2. , 1.4, 2.1,
       1.2, 1.3, 1.5, 1.1])

In [14]:
products.no_of_ratings

0         2,255
1         2,948
2         1,206
3            69
4           630
          ...  
551580        9
551581        2
551582        1
551583       37
551584        5
Name: no_of_ratings, Length: 551585, dtype: object

In [15]:
# Filters products with numeric ratings
products = products[products['no_of_ratings'].astype(str).str[0].str.isdigit()]

# Converts the 'no_of_ratings' column to float after removing commas
products["no_of_ratings"] = products["no_of_ratings"].str.replace(',', '').astype(float)

### Check NaN values

In [16]:
products.isnull().sum()

name                  0
main_category         0
sub_category          0
image                 0
link                  0
ratings               0
no_of_ratings         0
discount_price    34595
actual_price       6761
dtype: int64

In [17]:
products = products.dropna(subset=['actual_price','discount_price'])

### Creating `Discount` related columns 

In [18]:
# Make column with discount net value and discounting percent 
products['discount_value'] = products['actual_price'] - products['discount_price']
products['discounting_percent'] = 1 - products['discount_price']/products['actual_price']

In [19]:
products['brand'] = products['name'].str.split(' ').str[0]
print(products['brand'])
products['brand'] = products['brand'].str.lower().str.capitalize()

0              Lloyd
1                 LG
2                 LG
3                 LG
4            Carrier
             ...    
551579            BE
551580        Adidas
551581       Redwolf
551582       Redwolf
551584    Mothercare
Name: brand, Length: 334963, dtype: object


In [23]:
dataset = pd.merge(products, df, on='brand', how='inner')

In [25]:
dataset.to_csv('final_dataset.csv')